In [ ]:
from typing import Literal
# Bib joblib
from joblib import delayed,Parallel
# Numpy
import numpy as np
# Bib skimage 
import skimage
# Bib glob
import glob
# GLCM
from DescritoresDeImagem.glcm import glcm
# LBP
from DescritoresDeImagem.lbp import lbp
#LPQ
from DescritoresDeImagem.lpq import lpq

# Standart Scale, SVC and KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay


In [154]:
def  glcmimageapply(imgpath,distances=[1,3,5],angles=np.deg2rad([0,90,180,270]))-> np.ndarray[tuple[int],np.dtype[np.float64]]:
    """
    Args
    
    img (np.ndarray): 
        A 2D grayscale image array for which the GLCM is computed. The image 
        should be of type `np.ndarray` and must be 2-dimensional.
            
    distances (Union[List[int], np.ndarray]): 
        A list or array of pixel pair distances for which the GLCM is computed. 
        Default values are [1, 3, 5].
        
    angles (Union[List[float], np.ndarray]): 
        A list or array of angles (in radians) defining the direction for GLCM computation. 
        Default values are the radian equivalents of [0, 90, 180, 270] degrees.
    """
    imgopen=skimage.io.imread(imgpath,as_gray=True) # open image in as gray 
    img_info=glcm(imgopen,distances=distances,angles=angles) # aplly glcm in image open
          
    return img_info 


In [155]:


def lpqimageapply(imgpath, winSize: int = 7,decorr: int = 1,mode: str = 'nh'):
    """
    Applies the LPQ (Local Phase Quantization) method to an image.

    Parameters:
        imgpath (str): Path to the image file.
        winSize (int, optional): Window size for LPQ. Default is 7.
        decorr (int, optional): Decorrelation parameter for LPQ. Default is 1.
        mode (str, optional): Mode for LPQ ('nh' for normalized histogram). Default is 'nh'.

    Returns:
        np.ndarray: Feature vector or matrix produced by the LPQ method.
    """
    imgopen=skimage.io.imread(imgpath,as_gray=True) # open image in as gray 
    img_info=lpq(imgopen,winSize=winSize,mode=mode,decorr=decorr) # aplly glcm in image open
          
    return img_info 


In [156]:
def lbpimageaplly(imgpath,P: int = 8,R: int = 2,method: Literal['default', 'ror', 'uniform', 'nri_uniform', 'var'] = 'nri_uniform'):
    """
    Applies the LBP (Local Binary Patterns) method to an image.

    Parameters:
        imgpath (str): Path to the image file.
        P (int, optional): Number of circularly symmetric neighbor points. Default is 8.
        R (int, optional): Radius of the circle. Default is 2.
        method (Literal, optional): Method to extract LBP. Options include:
            - 'default': Basic LBP.
            - 'ror': Rotation invariant.
            - 'uniform': Uniform patterns only.
            - 'nri_uniform': Non-rotation-invariant uniform patterns.
            - 'var': Variance-based patterns. Default is 'nri_uniform'.

    Returns:
        np.ndarray: The resulting LBP image or feature vector.
    """
    
    imgopen=skimage.io.imread(imgpath,as_gray=True) # open image in as gray 
    
    img_info=lbp(image=imgopen,P=P,R=R,method=method) # aplly glcm in image open
          
    return img_info


In [157]:
def writendarry(filename:str,array:np.ndarray,delimiter="|")->None:
    with open(file=filename,mode="w") as file:
        for i in array:
            file.writelines(np.array2string(i,separator=","))
            file.write(delimiter)
            

In [158]:
def readfilendarray(filename:str,delimiter="|",data="f")->np.ndarray:
    if data=="f":
        array=[]
        with open(file=filename,mode="r") as f:
            lines=f.read().split(delimiter)
            for datastr in lines:
                data=datastr.replace("\n","").replace(" ","").strip("[]").split(",")
                arr=[float(i) for i in data if i.strip()]
                array.append(arr)
            array.pop()
            return np.array(array)
    elif data=="s":
        with open(file=filename,mode="r") as f:
            lines=f.read().replace("'","").split(delimiter)
            lines.pop()
            return np.array(lines)
            

In [159]:
# Number Threads
JOBS=12
# Use descriptor
DESCRIPTOR="LBP" #LPQ,LBP,GLCM
#Run Cell
RUN=True
 
pathimageitetor=glob.glob("BaseDeDados/*.bmp")

if DESCRIPTOR=="LPQ" and RUN: 
    datasetlpq=Parallel(n_jobs=JOBS)(delayed(lbpimageaplly)(i) for i in pathimageitetor)
    writendarry(filename="DadosExtraidos/X_TreinoLPQ.txt",array=datasetlpq)
elif DESCRIPTOR=="LBP" and RUN:
    datasetlbq=Parallel(n_jobs=JOBS)(delayed(lpqimageapply)(i) for i in pathimageitetor)
    writendarry(filename="DadosExtraidos/X_TreinoLBP.txt",array=datasetlbq)
elif DESCRIPTOR=="GLCM" and RUN:
    datasetlbq=Parallel(n_jobs=JOBS)(delayed(glcmimageapply)(i) for i in pathimageitetor)
    writendarry(filename="DadosExtraidos/X_TreinoGLCM.txt",array=datasetlbq)
if RUN:
    y_result=[]
    for i in pathimageitetor:
        y_result.append(i.replace("BaseDeDados/","").replace(".bmp",""))
    writendarry("DadosExtraidos/Y_Resultado.txt",np.array(y_result))


In [ ]:
X=readfilendarray(filename="DadosExtraidos/X_TreinoLFP.txt")

y=readfilendarray(filename="DadosExtraidos/Y_Resultado.txt",data="s")

modelo=None

FileNotFoundError: [Errno 2] No such file or directory: 'DadosExtraidos/X_TreinoLF.txt'

In [ ]:
ConfusionMatrixDisplay.from_estimator(modelo, X, y)

In [161]:
print(classification_report(y,y))

              precision    recall  f1-score   support

          a1       1.00      1.00      1.00         1
         a10       1.00      1.00      1.00         1
        a100       1.00      1.00      1.00         1
        a101       1.00      1.00      1.00         1
        a102       1.00      1.00      1.00         1
        a103       1.00      1.00      1.00         1
        a104       1.00      1.00      1.00         1
        a105       1.00      1.00      1.00         1
        a106       1.00      1.00      1.00         1
        a107       1.00      1.00      1.00         1
        a108       1.00      1.00      1.00         1
        a109       1.00      1.00      1.00         1
         a11       1.00      1.00      1.00         1
        a110       1.00      1.00      1.00         1
        a111       1.00      1.00      1.00         1
        a112       1.00      1.00      1.00         1
        a113       1.00      1.00      1.00         1
        a114       1.00    